# 2.2 - Linear Predictive Coding

<img style="float: right; margin:5px 0px 0px 10px" src="img/title.jpg" width="400">
Einführung...

## Inhalt  
<table style="width:256px; border: 1px solid black; display: inline-block">
    <tr>
        <td  style="text-align:right" width=64px><img src="img/1.jpg" style="float:left"></td>
        <td style="text-align:left" width=256px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#1'>
                1. 
            </a>
        </td>
    </tr>  
    <tr>
        <td style="text-align:right"><img src="img/2.jpg" style="float:left"></td>
        <td style="text-align:left" width=128px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#2'>
                2. 
            </a>
        </td>
    </tr>
    <tr>
        <td style="text-align:right"><img src="img/3.jpg" style="float:left"></td>
        <td style="text-align:left" width=128px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#3'>
                3. 
            </a>
        </td>
    </tr>
</table>

----

<a id='1'></a>
<div>
    <img src="img/1.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        1.  
    </h2>
</div>

Beschreibungen...

----

<a id='2'></a>
<div>
    <img src="img/2.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        2.  
    </h2>
</div>

Beschreibungen...

----

<a id='3'></a>
<div>
    <img src="img/3.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        3.  
    </h2>
</div>

Beschreibungen...

----

### References

1. Titelbild von [TITLE]()  
2. [SOURCE]()